In [1]:
from eodhd import APIClient
import requests 
import pandas as pd
import time
from tqdm import tqdm
import os


api = "64d77f6d3a60a5.24835840"

C:\Users\Soren\AppData\Roaming\Python\Python310\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (2.0.3) or chardet (None)/charset_normalizer (3.1.0) doesn't match a supported version!
  warnings.warn(


In [7]:
# First define the log function to gather the log information
def log(response,logfile, stock, offset,output_path=os.getcwd()):
    # Open or create the csv file
    if os.path.isfile(logfile): #If the log file exists, open it and allow for changes     
        log = open(logfile,'a')
    else: #If the log file does not exist, create it and make headers for the log variables
        log = open(logfile,'w')
        header = ['timestamp','status_code','length','Ticker','Offset']
        log.write(';'.join(header) + "\n") #Make the headers and jump to new line
        
    # Gather log information
    status_code = response.status_code #Status code from the request result
    timestamp = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())) #Local time
    length = len(response.text) #Length of the HTML-string
    
    # Open the log file and append the gathered log information
    with open(logfile,'a') as log:
        log.write(f'{timestamp};{status_code};{length};{stock};{offset}' + "\n") #Append the information and jump to new line

In [9]:
# Opening the csv file named NASDAQ 100, reading it and making it into a dataframe

# Change the current working directory to the folder where the script is located
os.chdir('C:/Users/Soren/Documents/GitHub/IntroSocial23/Exam')

# Get the current working directory
current_directory = os.getcwd()

# Construct the path to the CSV file
csv_file_path = os.path.join(current_directory, 'Data collection','Data','NASDAQ 100.csv')

# Read the CSV file and create a DataFrame
df_companies = pd.read_csv(csv_file_path, delimiter=';', encoding='utf-8')

# make df_companies into a dataframe
df_companies = pd.DataFrame(df_companies)

In [11]:
# Get all news

# Dates
start_date = '2020-01-01'
end_date = '2023-07-31'

# number of articles pr. stock
n_news = 3
stock_list = []
stock_list = df_companies['Symbol']
company_names = []
company_names = df_companies['Company']

logfile = 'log.csv'

def get_all_news(stocks, start_date, end_date, n_news, api_key, offset = 0):
    # create empty dataframe
    all_news = pd.DataFrame()
    for stock in tqdm(stocks, desc="Fetching News", unit="stock"):
        # url for the api call
        url = f'https://eodhistoricaldata.com/api/news?api_token={api_key}&s={stock}&limit={n_news}&offset={offset}&from={start_date}&to={end_date}'
        # get the json from the api call
        response = requests.get(url)
        news_json = response.json()
        
        # create dataframe from json
        df_news = pd.DataFrame.from_dict(news_json)
        # filter the dataframe
        df_news = df_news[['date','title','content','symbols','sentiment']]
        
        # a new column with the company name 
        df_news['company'] = company_names[stock_list == stock].values[0]
        
        # concat df_news onto all_news
        all_news = pd.concat([all_news, df_news], ignore_index = True)
        
        # Log the response
        log(response, logfile, stock, offset)
        
        # save the dataframe
        all_news.to_csv('df_all_news.csv', index=False)
    return all_news 

df_all_news = get_all_news(stock_list, start_date, end_date, n_news, api)


df_all_news



Fetching News: 100%|██████████| 101/101 [01:45<00:00,  1.04s/stock]


,date,title,content,symbols,sentiment,company
0,2023-07-31T23:54:16+00:00,"Asian stocks push higher on tech strength, Chi...",Investing.com -- Most Asian stocks rose on Tue...,"[0700.HK, 2007.HK, 9888.HK, 9988.HK, AAPL.US, ...","{'polarity': 0.992, 'neg': 0.066, 'neu': 0.794...",Apple Inc
1,2023-07-31T21:07:35+00:00,"Dow Jones Futures: Microsoft, Tesla At Key Sup...",Dow Jones futures: Microsoft and Tesla stock c...,"[AAPL.MX, AAPL.US, AAPL34.SA, ALGM.US, APC.F, ...","{'polarity': 0, 'neg': 0, 'neu': 1, 'pos': 0}",Apple Inc
2,2023-07-31T21:01:39+00:00,The 2023 stock market rally got a lot healthie...,"July has come and gone for investors, bringing...","[AAPL.MX, AAPL.US, AAPL34.SA, ABEA.F, ABEA.XET...","{'polarity': 0.991, 'neg': 0.026, 'neu': 0.88,...",Apple Inc
3,2023-07-31T23:54:16+00:00,"Asian stocks push higher on tech strength, Chi...",Investing.com -- Most Asian stocks rose on Tue...,"[0700.HK, 2007.HK, 9888.HK, 9988.HK, AAPL.US, ...","{'polarity': 0.992, 'neg': 0.066, 'neu': 0.794...",Microsoft Corp
4,2023-07-31T21:31:58+00:00,Arista Slays Bears As Earnings Beat Amid Micro...,Arista Networks reported second-quarter earnin...,"[117.F, ANET.US, FB2A.F, FB2A.XETRA, MSF.F, MS...","{'polarity': 0, 'neg': 0, 'neu': 1, 'pos': 0}",Microsoft Corp
...,...,...,...,...,...,...
298,2023-07-28T17:00:55+00:00,11 Best Logistics Stocks to Buy,"In this article, we discuss the 11 best logist...","[AMZN.US, AQY.F, ARCB.US, BABA.US, CSX.US, CSX...","{'polarity': 0.999, 'neg': 0.013, 'neu': 0.882...",JD.com Inc ADR
299,2023-07-27T17:49:42+00:00,3 Sleeper Growth Stocks Set to Deliver Explosi...,Many growth stocks that haven’t aligned with W...,"[013A.F, 5ZM.F, 9618.HK, BABA.US, JD.US, JDCMF...","{'polarity': 0.998, 'neg': 0.035, 'neu': 0.821...",JD.com Inc ADR
300,2023-07-28T17:33:21+00:00,Why Lucid Stock Jumped Friday,Established automakers are finding out it's ha...,"[F.MX, F.US, FDMO34.SA, FMC1.F, FMC1.XETRA, LC...","{'polarity': 0.844, 'neg': 0.018, 'neu': 0.845...",Lucid Group Inc
301,2023-07-27T12:36:00+00:00,Proceed With Caution When Considering These 5 ...,Everything would have to go right for the comp...,"[CCL.LSE, CCL.US, CUK.US, CUKPF.US, CVC1.F, LC...","{'polarity': 0.511, 'neg': 0, 'neu': 0.938, 'p...",Lucid Group Inc


In [11]:
# save df_all_news to csv
df_all_news.to_csv('df_all_news.csv', index=False)